In [ ]:
# Installation of packages and embedding
import sys
# Import spaCy ,load model
!{sys.executable} -m spacy download en_core_web_sm

     |████████████████████████████████| 10.4 MB 23.5 MB/s 
     |████████████████████████████████| 184 kB 71.7 MB/s 
     |████████████████████████████████| 1.0 MB 53.3 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.1
    Not uninstalling srsly at /shared-libs/python3.7/py/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'srsly'. No files were found to uninstall.
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.6
    Not uninstalling catalogue at /shared-libs/python3.7/py/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'catalogue'. No files were found to uninstall.
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.10
    Not uninstalling thinc at /shared-libs/python3.7/py/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'thinc'. No files were found to uninstall.
  Attempting uninstall: spacy
    Found existi

In [ ]:
import spacy
nlp=spacy.load("en_core_web_sm")
nlp.pipe_names

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [ ]:
!pip install -U spacy==2.3.6

     |████████████████████████████████| 10.4 MB 35.5 MB/s 
     |████████████████████████████████| 1.0 MB 72.8 MB/s 
     |████████████████████████████████| 184 kB 85.2 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.1
    Not uninstalling srsly at /shared-libs/python3.7/py/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'srsly'. No files were found to uninstall.
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.6
    Not uninstalling catalogue at /shared-libs/python3.7/py/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'catalogue'. No files were found to uninstall.
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.10
    Not uninstalling thinc at /shared-libs/python3.7/py/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'thinc'. No files were found to uninstall.
  Attempting uninstall: spacy
    Found existi

In [ ]:
# Adding the built-in textcat component to the pipeline.
textcat=nlp.create_pipe( "textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"})
nlp.add_pipe(text_cat, last=True)
nlp.pipe_names

ConfigValidationError: 

Config validation error

textcat -> architecture        extra fields not permitted
textcat -> exclusive_classes   extra fields not permitted

{'nlp': <spacy.lang.en.English object at 0x7fda7e5ca610>, 'name': 'textcat', 'architecture': 'simple_cnn', 'exclusive_classes': True, 'model': {'@architectures': 'spacy.TextCatEnsemble.v2', 'linear_model': {'@architectures': 'spacy.TextCatBOW.v2', 'exclusive_classes': True, 'ngram_size': 1, 'no_output_layer': False}, 'tok2vec': {'@architectures': 'spacy.Tok2Vec.v2', 'embed': {'@architectures': 'spacy.MultiHashEmbed.v2', 'width': 64, 'rows': [2000, 2000, 1000, 1000, 1000, 1000], 'attrs': ['ORTH', 'LOWER', 'PREFIX', 'SUFFIX', 'SHAPE', 'ID'], 'include_static_vectors': False}, 'encode': {'@architectures': 'spacy.MaxoutWindowEncoder.v2', 'width': 64, 'window_size': 1, 'maxout_pieces': 3, 'depth': 2}}}, 'threshold': 0.5, '@factories': 'textcat'}

In [ ]:
# Adding the labels to textcat
textcat.add_label("POSITIVE")
textcat.add_label("NEGATIVE")

1

In [ ]:
# Import dataset and pandas
import pandas as pd
raw_trainDF = pd.read_csv("/work/covid_sentiment/data/coronavirus_tweet_raw/Corona_NLP_test.csv")
raw_testDF = pd.read_csv("/work/covid_sentiment/data/coronavirus_tweet_raw/Corona_NLP_test.csv")
raw_trainDF.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [ ]:
trainDF = raw_trainDF
testDF = raw_testDF

In [ ]:
def label(row):
    if (row == "Extremely Negative") | (row == "Negative"): output = 0
    elif (row == "Extremely Positive") | (row == "Positive"): output = 1
    else: output = 0
    return output
trainDF['encoded'] = trainDF.Sentiment.apply(lambda row: label(row))
testDF['encoded'] = testDF.Sentiment.apply(lambda row: label(row))


In [ ]:
# Converting the dataframe into a list of tuples
trainDF['tuples'] = trainDF.apply(lambda row: (row['OriginalTweet'],row['encoded']), axis=1)
testDF['tuples'] = testDF.apply(lambda row: (row['OriginalTweet'],row['encoded']), axis=1)

train = trainDF['tuples'].tolist()
test = testDF['tuples'].tolist()
train[:10]

[('TRENDING: New Yorkers encounter empty supermarket shelves (pictured, Wegmans in Brooklyn), sold-out online grocers (FoodKick, MaxDelivery) as #coronavirus-fearing shoppers stock up https://t.co/Gr76pcrLWh https://t.co/ivMKMsqdT1',
  0),
 ("When I couldn't find hand sanitizer at Fred Meyer, I turned to #Amazon. But $114.97 for a 2 pack of Purell??!!Check out how  #coronavirus concerns are driving up prices. https://t.co/ygbipBflMY",
  1),
 ('Find out how you can protect yourself and loved ones from #coronavirus. ?',
  1),
 ('#Panic buying hits #NewYork City as anxious shoppers stock up on food&amp;medical supplies after #healthcare worker in her 30s becomes #BigApple 1st confirmed #coronavirus patient OR a #Bloomberg staged event?\r\r\n\r\r\nhttps://t.co/IASiReGPC4\r\r\n\r\r\n#QAnon #QAnon2018 #QAnon2020 \r\r\n#Election2020 #CDC https://t.co/29isZOewxu',
  0),
 ('#toiletpaper #dunnypaper #coronavirus #coronavirusaustralia #CoronaVirusUpdate #Covid_19 #9News  #Corvid19 #7NewsMelb #dun

In [ ]:
import random

def load_data(limit=0, split=0.8):
    train_data=train
    # Shuffle the data
    random.shuffle(train_data)
    texts, labels = zip(*train_data)
    # get the categories for each review
    cats = [{"POSITIVE": bool(y) , "NEGATIVE": not bool(y)} for y in labels]

    # Splitting the training and evaluation data
    split = int(len(train_data) * split)
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])


In [ ]:

# Calling the load_data() function 
(train_texts, train_cats), (dev_texts, dev_cats) = load_data()

# Processing the final format of training data
train_data = list(zip(train_texts,[{'cats': cats} for cats in train_cats]))
train_data[:10]

[('Stop #frenzybuying. You don\'t need most of what you\'ve been hoarding. This is proof that #boston\'s "norm" of me-first selfishness at the expense of others is more contagious than #coronapocalypse #CV19 #coronavirus #Covid_19 #COVID19 #COVID #marALagoVirus #trumpPlague #COVID?19 https://t.co/xxLWYK4TrY',
  {'cats': {'POSITIVE': False, 'NEGATIVE': True}}),
 ("Teachers, Loom is a great tool for recording and sharing videos with comments.\r\r\n\r\r\n#teachers #education #Covid_19 \r\r\n\r\r\n@useloom is there a way I can pay double per month while you're running these adjusted prices? Happy to if so.\r\r\n\r\r\nhttps://t.co/gRUnBAboIO",
  {'cats': {'POSITIVE': True, 'NEGATIVE': False}}),
 ('we can all get a little hazy or forgetful under pressure. with all the stress regarding COVID-19, don\x92t forget to stock up for your pets. buy some extra food, get extra prescription medication, extra toys for indoor and yard exercise. if you\x92re under quarantine, they are too',
  {'cats': {'P

In [ ]:
def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if label == "NEGATIVE":
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.0
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.0
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}


#("Number of training iterations", "n", int))
n_iter=10

In [ ]:
from spacy.util import minibatch, compounding
# Disabling other components
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.create_optimizer()
    
    print("Training the model...")
    print('{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('LOSS', 'P', 'R', 'F'))

    # Performing training
    for i in range(n_iter):
        losses = {}
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)

      # Calling the evaluate() function and printing the scores
        with textcat.model.use_params(optimizer.averages):
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print('{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}'  
              .format(losses['textcat'], scores['textcat_p'],
                      scores['textcat_r'], scores['textcat_f']))


ValueError: [E955] Can't find table(s) lexeme_norm for language 'en' in spacy-lookups-data. Make sure you have the package installed or provide your own lookup tables if no default lookups are available for your language.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=36980032-e74f-4047-828e-e2329ad1a610' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>